In [1]:
import os
import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
root = '../../Assets/101_ObjectCategories'
exclude = ['BACKGROUND_Google', 'Motorbikes', 'airplanes', 'Faces_easy', 'Faces']
train_split, val_split = 0.7, 0.15

categories = [x[0] for x in os.walk(root) if x[0]][1:]
categories = [c for c in categories if c not in [os.path.join(root, e) for e in exclude]]

print(categories)

['../../Assets/101_ObjectCategories\\accordion', '../../Assets/101_ObjectCategories\\anchor', '../../Assets/101_ObjectCategories\\ant', '../../Assets/101_ObjectCategories\\barrel', '../../Assets/101_ObjectCategories\\bass', '../../Assets/101_ObjectCategories\\beaver', '../../Assets/101_ObjectCategories\\binocular', '../../Assets/101_ObjectCategories\\bonsai', '../../Assets/101_ObjectCategories\\brain', '../../Assets/101_ObjectCategories\\brontosaurus', '../../Assets/101_ObjectCategories\\buddha', '../../Assets/101_ObjectCategories\\butterfly', '../../Assets/101_ObjectCategories\\camera', '../../Assets/101_ObjectCategories\\cannon', '../../Assets/101_ObjectCategories\\car_side', '../../Assets/101_ObjectCategories\\ceiling_fan', '../../Assets/101_ObjectCategories\\cellphone', '../../Assets/101_ObjectCategories\\chair', '../../Assets/101_ObjectCategories\\chandelier', '../../Assets/101_ObjectCategories\\cougar_body', '../../Assets/101_ObjectCategories\\cougar_face', '../../Assets/101_Obje

In [3]:
# helper function to load image and return it and input vector
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

In [4]:
data = []
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames 
              in os.walk(category) for f in filenames 
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    for img_path in images:
        img, x = get_image(img_path)
        data.append({'x':np.array(x[0]), 'y':c})

random.shuffle(data)
        
# count the number of classes
num_classes = len(categories)
num_samples = len(data)
num_classes, num_samples

(97, 6209)

In [5]:
idx_train = int(train_split * num_samples)
idx_val = int((train_split + val_split) * num_samples)
train = data[0:idx_train]
val = data[idx_train:idx_val]
test = data[idx_val:]

In [6]:
len(train), len(val), len(test)

(4346, 931, 932)

In [7]:
x_train = np.array([t['x'] for t in train])
x_val = np.array([t['x'] for t in val])
x_test = np.array([t['x'] for t in test])

y_train = np.array([t['y'] for t in train])
y_val = np.array([t['y'] for t in val])
y_test = np.array([t['y'] for t in test])

In [8]:
y_train.shape

(4346,)

In [9]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [10]:
y_train.shape

(4346, 97)

In [13]:
vgg = keras.applications.VGG16(weights='imagenet',include_top=True)
vgg.summary()

553467904/553467096 [==============================] - 303s 1us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_

In [14]:
inp = vgg.input
new_classification_layer = Dense(num_classes, activation='softmax')
out = new_classification_layer(vgg.layers[-2].output)
model_new = Model(inp, out)
model_new.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [26]:
for l,layer in enumerate(model_new.layers[0:-1]):
    layer.trainable = False

for l,layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True
    
model_new.layers

In [27]:
model_new.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
model_new.fit(x_train, y_train,
              batch_size=128,
              epochs=100,
              verbose=1,
              validation_data=(x_val, y_val))

Train on 4346 samples, validate on 931 samples
Epoch 1/100
4346/4346 [==============================] - 26s 6ms/step - loss: 1.5125 - acc: 0.6726 - val_loss: 0.6269 - val_acc: 0.8228
Epoch 2/100
4346/4346 [==============================] - 20s 5ms/step - loss: 0.1170 - acc: 0.9662 - val_loss: 0.5229 - val_acc: 0.8528
Epoch 3/100
4346/4346 [==============================] - 20s 5ms/step - loss: 0.0299 - acc: 0.9961 - val_loss: 0.5162 - val_acc: 0.8657
Epoch 4/100
4346/4346 [==============================] - 20s 5ms/step - loss: 0.0185 - acc: 0.9965 - val_loss: 0.5194 - val_acc: 0.8614
Epoch 5/100
4346/4346 [==============================] - 20s 5ms/step - loss: 0.0115 - acc: 0.9982 - val_loss: 0.4945 - val_acc: 0.8679
Epoch 6/100
4346/4346 [==============================] - 20s 5ms/step - loss: 0.0066 - acc: 0.9998 - val_loss: 0.4952 - val_acc: 0.8700
Epoch 7/100
4346/4346 [==============================] - 20s 5ms/step - loss: 0.0051 - acc: 1.0000 - val_loss: 0.4931 - val_acc: 0.8679
E

KeyboardInterrupt: 